# Air Entrainment Analysis Automatized


## by $\Re\in\Upsilon\sqcap\alpha$

---
<div style="text-align: right"> 
author: R.G. Ramirez de la T.

<div style="text-align: right">
start date: 04-Sep-2018 (extracted from Photron Make Data)

<div style="text-align: right">
last modification: 14-Sep-2018
</div>


In [192]:
import cv2
import os, fnmatch
import math
import glob
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff

In [193]:
import scipy
from scipy import signal
%matplotlib inline

## Functions for lsv files and flow data

In [3]:
def lvm_unpack(name,rows=25):
    """Extract time and flow data from the lab view .lvm files as output of the 
    Flowmeter Application developed in the lab. It includes the conevrsion from Volts to L/min
    as in 10V = 3L/min.
    
    This function reads the name of the lvm file unpack the content in a matrix 
    and converts it to type float.
    
    input:
        name: the name of the file (with extension if it is in another folder)
        rows: the the number of rows that should be skipped from the file.

    output:
        2 vectors:
            time: contains the time data
            flow: contains the flow data
        
    example:

    name='FlowMeterData_001.lvm'
    t,flow = lvm_unpack(name)
    
    see also: 
    
    numpy.loadtxt, numpy.char, numpy.array
 
    TODO: include as a parameter the conversion V to L/min, store only a portion
            of the file.
    NOTE: 
    """
    thefile = open(name, 'r')
    x = np.loadtxt(thefile, dtype=np.str, unpack=True,skiprows=rows)
    x = np.char.replace(x, ',','.')
    x = np.array(x,dtype = np.float)
    t = x[0]
    flow = 3.*x[1]/10.
    return t, flow

In [195]:
def Mean_Flow(flow):
    """Calculate the mean flow of a flow vector.
    
    If the flow vector includes a big curve from zero to the stable flow,
    it only considers the stable part to make the calculation of the mean.
    
    input:
        flow: the input flow.

    output:
        the mean flow
        
    example:

    t = linspace(0,2,0.1)
    flow = log(t)
    average=Mean_Flow(flow)
    
    see also: 
    
    numpy.mean, numpy.where.
 
    NOTE: 
    """
    deri = np.mean(flow)
    pos = np.where(flow >deri)
    save = 0
    for i in range(len(pos[0])):
        if save == 0:
            if pos[0][i+1]-pos[0][i] >1:
                continue
            elif pos[0][i+1]-pos[0][i] == 1:
                save = pos[0][i]
    aver = np.mean(flow[save:])
    return aver

## Functions for image processing

In [196]:
def new_image(img):
    """make a black and white image with only the edges.
    
    Takes a grayscale image and using adaptative threshold, detects the edges and
    gives a new black an white image with only the edges.
    
    input:
        img:grayscale image.

    output:
        th2: new grayscale image with only black and white values.
        
    example:

    image = cv2.imread(file)
    edges=new_image(image)
    
    see also: 
    
    cv2.mediaBlur, cv2.threshold, cv2.adaptativeThreshold.
    
    TODO: make the threshold value a parameter in the function.
    NOTE: 
    """
    img_blur = cv2.medianBlur(img,7)
    ret,th1 = cv2.threshold(img_blur,180,255,cv2.THRESH_BINARY)
    th2 = cv2.adaptiveThreshold(img_blur,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY_INV,11,2)
    return th2

In [197]:
def extract_line(image):
    """Find the edges of the jet images and saves them into vectors.
    
    Takes a grayscale image of the water jet and using the Sobel algorithm
    for edge detection finds the left and the right edges of the jet.
    
    input:
        img:grayscale image.

    output:
        sobelx: the Sobel gradiant 
        right_line: the right edge
        left_line: the left edge
        
    example:

    image = cv2.imread(file)
    grad,right,left=extract_line(image)
    
    see also: 
    
    cv2.mediaBlur, cv2.threshold, cv2.adaptativeThreshold.
    
    TODO: 
    NOTE: 
    """
    #The image is Gray scale, taking the first channel is OK
    image = image[:,:]
    #Calculating the gradian over the y axis of the image
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=31)
    #Finaly, ge the highest value of it for each colums
    right_line = np.argmax(sobelx[:, sobelx.shape[1]//2:], axis=1)
    right_line = np.array([right_line + sobelx.shape[1]//2, image.shape[0]-np.arange(image.shape[0])])
    left_line = np.argmin(sobelx[:, :sobelx.shape[1]//2], axis=1)
    left_line = np.array([left_line, image.shape[0]-np.arange(image.shape[0])])
    return sobelx, right_line, left_line

In [198]:
def correct_edges(line):
    """Takes a vector with the edges on an image and checks the continuity of the data. 
    If it has big discontinuities (>5 pixel jumps), the data es corrected.
    
    Takes a vector and check that its content varies smoothly, if it has 
    any discontinuities or big jumps, this data is replaced with previous
    nearest value in the array.
    
    input:
        line:a vector preferably with integer entries.

    output:
        line: returns the corrected line.
        
    example:

    smooth_line = corrected_edges(line)
    
    
    TODO: make discontinuity value a parameter in the function. Better example in doc
    NOTE:This function is thought for "pixeled" vectors where the edge detection can 
        fail due to luminosity issues. It is not recomended for vectors with non-
        integer entries.
    """
    total = len(line)
    check =line[0]
    for i in range(1,total):
        dif = abs(check-line[i])
        if dif > 5:
            line[i] = check
        check = line[i]
    return line

In [199]:
def Ripple_Sobel(num,total = None,saving = False):
    """Make edges matrix of numbered sequences of images.
    
    Takes the number of the image sequence and use it to generate the name
    of the containing folder '../hoose jet/10-ago-18/8bit/FLM'+str(num)+'/'.
    This needs to be changed manually inside the function in case the sequences
    are contained in another folder. 
    Then it performs the edege detection and store the result in two matrixes
    If the total parameter is supplied, it only analizes total number of pictures.
    If saving parameter is supplied, it saves the data in .txt files called
    'sobel_right'+str(num)+'.txt' and 'sobel_left'+str(num)+'.txt'.
    
    input:
        num:integer value, number of the folder for the image sequence
        total: total of images to analyze. The default value is None, which means
            it will analyze all the images.
        saving: If True, generates the .txt data files for each edge. False by default.
        

    output:
        left: edge matrix for the left side, columns contain the postion of the edge 
            for each pixel in the length of the jet and the rows contain each 
            time step or image.
        right: edge matrix for the right side, columns contain the postion of the edge 
            for each pixel in the length of the jet and the rows contain each 
            time step or image.
        
    example:

    
    see also: 
    
    glob.glob, tiff.imread, numpy.zeros, numpy.savetxt,extract_line, correct_edges.
    
    TODO: make the folder a parameter in the function.
    NOTE: The saved files, when extracted with numpy.loadtxt, are transposed 
        compared to the direct output of this function, i.e. rows contains images
        and columns contains space coordinates.
    """
    #FILES AND NAMES
    #open folder where pics are contained
    dirPhot = '../hoose jet/10-ago-18/8bit/FLM'+str(num)+'/'
    #look for the right name in the sequence files
    pattern1 = "FLM"+str(num)+"_cutted*.tif"
    #sort names in right numericla order
    filesPhot=sorted(glob.glob(dirPhot+pattern1))
    #confirm the name of the first image by printing it
    print filesPhot[0]
    
    if saving == True:
    #make name for txt data file and open file
        txt_name_r = 'sobel_right'+str(num)+'.txt'
        txt_name_l = 'sobel_left'+str(num)+'.txt'
        data_file_r= open(txt_name_r,'w')
        data_file_l = open(txt_name_l,'w')
    
    #If total is given, analyze only that amount of images, otherwise take all
    if total == None:
        no_images = len(filesPhot)
    else:
        no_images = total
    
    #START ANALYSIS
    
    # Time count with i
    i = 0
    #Read one image to alocate space in the data matrix 
    first = tiff.imread(filesPhot[0])
    length = first.shape[0]
    
    right = np.zeros([length,no_images])
    left = np.zeros([length,no_images])
    dum1 = np.zeros(length)
    dum2 = np.zeros(length)
    
    #Loop for eache image
    for arc in filesPhot[0:no_images]:
        im = tiff.imread(arc)
        sob,l1,r1 = extract_line(im)
        dummy_l = l1[0,:]
        dummy_r = r1[0,:]
        lefty = correct_edges(dummy_l)
        righty = correct_edges(dummy_r)
        left[:,i] = lefty
        right[:,i] = righty
            
    #Time advance
        i +=1
    if saving == True:
    #Save data in files 
        np.savetxt(txt_name_l,left,fmt='%1.2f')
        np.savetxt(txt_name_r,right,fmt='%1.2f')
        data_file_l.close()
        data_file_r.close()
    return left, right

In [200]:
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y[(window_len/2-1):-(window_len/2)]

## Analysis Functions

Until here we have saved the pixel data in to files to start reducing the amount of data storage.
Now we make new files removing the zero crossing and smooting the signal.

In [201]:
def Smoothed_waves(num,total = None,win=50,saving=True):
    """Make smoothed edges matrixes out of pixeled edges matrixes.If saving is True,
        saves the result in a txt file.
    
    Takes the number of the txt file, and use it to generate the name
    of the file as 'sobel_right'+str(num)+'.txt' and 'sobel_left'+str(num)+'.txt'
    This needs to be changed manually inside the function in case the files
    are contained in another folder. 
    Then it smooths the edges data and store the result in two new matrixes.
    If the total parameter is supplied, it only analizes total number of edges.
    If saving parameter is supplied, it saves the data in .txt files called
    'smooth_right'+str(num)+'.txt' and 'smooth_left'+str(num)+'.txt'.
    
    input:
        num:integer value, number of the txt file
        total: total of edges to smooth. The default value is None, which means
            it will analyze all the edges in the matrix.
        win: Size of the window to smooth the data. 50 by default.
        saving: If True, generates the .txt data files for each edge. False by default.
        

    output:
        left: smoothed edge matrix for the left side, rows contain the postion of the edge 
            for each pixel in the length of the jet and the columns contain each 
            time step or image.
        right: smoothed edge matrix for the right side, rows contain the postion of the edge 
            for each pixel in the length of the jet and the columns contain each 
            time step or image.
        
    example:

    
    see also: 
    
    numpy.loadtxt, numpy.zeros, numpy.savetxt, smooth.
    
    TODO: make the folder a parameter in the function.
    NOTE: The saved files, when extracted with numpy.loadtxt, are transposed 
        compared to the direct output of this function, i.e. rows contains images
        and columns contains space coordinates.
    """
    #FILES AND NAMES
        
    if saving==True:
    #make name for txt new data file and open file
        txt_name_r = 'smooth_right'+str(num)+'.txt'
        txt_name_l = 'smooth_left'+str(num)+'.txt'
        data_file_r= open(txt_name_r,'w')
        data_file_l = open(txt_name_l,'w')

    #open data files and extract data
    name_r = 'sobel_right'+str(num)+'.txt'
    name_l = 'sobel_left'+str(num)+'.txt'
    data_r = np.loadtxt(name_r,unpack=True)
    data_l = np.loadtxt(name_l,unpack=True)
    
    
    #If total is given, analyze only that amount of signals, otherwise take all
    if total == None:
        no_images = data_l.shape[0]
    else:
        no_images = total
    
    #START ANALYSIS

    # Alocate space in the data matrix
    left = np.zeros(data_l.shape)
    right = np.zeros(data_r.shape)
    
    
    #Loop for each image(time step)
    for i in range(no_images):
        wave_l = smooth(data_l[i,:],window_len=win,window='hanning')
        wave_r = smooth(data_r[i,:],window_len=win,window='hanning')
        left[i,:] =wave_l
        right[i,:] =wave_r
    if saving==True:
    #Save data in files 
        np.savetxt(txt_name_l,left,fmt='%1.3f')
        np.savetxt(txt_name_r,right,fmt='%1.3f')
        data_file_l.close()
        data_file_r.close()
    return left, right

In [202]:
def Zero_cross(data,name='curve'):
    """Takes a matrix and calculates the mean along the rows. If the 
    parameter name is 'line', it fits a straight line to the mean.
        
    Takes a matrix that contains the edges or waves for each time step(image)
    in each row, then it calculates the temporal mean for each point in the length.
    This mean curve will be consider the zero crossing of the waves.
    If the name is 'line' it fits a straight line to the resultant mean.
    
    input:
        data:matrix that contian waves or ripples in the rows.
        name: name of the type of zero-crossing desired. Options are:
            'line' for fitting a straight line, 'curve' for using the mean calculated.
            By default this options is 'curve'.

    output:
        crossing: returns zero-crossing curve along the jet.
        
    example:
        x = numpy.linspace(0,2*np.pi,100)
        y = numpy.sin(x)
        crossing = Zero_cross(y)  
    
    see also:
        numpy.mean, numpy.arange, numpy.polyfit, numpy.poly1d
    
    TODO: ??
    NOTE:
    """
    if name == 'line':
        average = np.mean(data,axis=0)
        x = np.arange(data.shape[1])
        coef = np.polyfit(x,average,1)
        line = np.poly1d(coef)
        return line(x)
    if name == 'curve':
        return np.mean(data,axis=0)

In [203]:
def substract_zero_crossing(data):
    """Takes a matrix containing waves or ripples and substract the zero-crossing.
    Then all the troughs are negative and all the crests are positive and the zero
    position is the hypothetical undisturbed surface.
        
    Takes a matrix that contains the edges or waves for each time step(image)
    in each row, then it calculates the temporal mean for each point in the length.
    This mean curve will be consider the zero crossing of the waves.
    If the name is 'line' it fits a straight line to the resultant mean.
    
    input:
        data:matrix that contian waves or ripples in the rows.

    output:
        new_data: returns the matrix with substracted zero-crossing.
        
    example:
        x = numpy.linspace(0,2*np.pi,100)
        y = 0.5*x+numpy.sin(x)
        crossing = substract_zero_crossing(y) 
    
    se also:
        numpy.zeros_like, Zero_cross
    
    TODO: ??
    NOTE: This function can only be used to calculate the total mean of the series
        if the zero crossing change in time, use the function:
        substract_zero_crossing_time
    """
    new_data = np.zeros_like(data)
    zeroc = Zero_cross(data)
    for i in range(data.shape[0]):
        new_data[i][:] = data[i][:]-zeroc
    return new_data

In [204]:
def substract_zero_crossing_time(data,size=100):
    """Takes a matrix containing waves or ripples and substract the zero-crossing.
    The zero crossing changes along the matrix taking samples of the specified size. 
    Then all the troughs are negative and all the crests are positive and the zero
    position is the hypothetical undisturbed surface.
        
    Takes a matrix that contains the edges or waves for each time step(image)
    in each row, then it calculates a mean every determined quantity of time steps
    for each point in the length. The quantity of time steps is determined by the 
    parameter size. Then substract this mean curve from all the data sets that 
    participated in the mean calculation.
    
    input:
        data:matrix that contain waves or ripples in the rows.
        size: size of the samples to take the mean and substract.

    output:
        new_data: returns the matrix with substracted zero-crossing.
        
    example:
        t = ??
        x = numpy.linspace(0,2*np.pi,100)
        y = 0.5*x+numpy.sin(x)
        crossing = substract_zero_crossing(y) 
    
    se also:
        numpy.zeros_like, Zero_cross
    
    TODO: finish example in doc
    NOTE: If the reminder of size/total_size is different than zero, the last mean will
    be calculated with the remaining timesteps.
    """
    new_data = np.zeros_like(data)
    N = data.shape[0]//size
    dif = abs(N*size-data.shape[0])
    for j in range(N-1):
        zeroc = Zero_cross(data[j*size:(j+1)*size,:])
        for i in range(j*size,(j+1)*size):
            new_data[i][:] = data[i][:]-zeroc
    if dif != 0:
        zeroc = Zero_cross(data[N*size:N*size+dif,:])
        for i in range(N*size,N*size+dif):
            new_data[i][:] = data[i][:]-zeroc
    return new_data

In [205]:
def Find_waves(wave,side,dist=40):
    """Takes a vector, finds the indexes of troughs and store them in a list.
        
    Takes a vector containing a wavy signal with zero crossings, finds the peaks 
    and, depending which side of the jet it is, calculates the minima or the maxima 
    as the throughs. It eliminates all the troughs which do not have an inbetween
    crest higher than the zero-crossing.
    
    input:
        wave:vector signal with zero-crossing data,i.e. minima<0 and maxima>0.
        side:specification on which side of the jet is been anlayzed. It is
            assumed that the water is at the negative side of the left edge and 
            to the positive side of the right edge. So, if left is selected the 
            signal is inverted in sign. 
        dist:distance between peaks, 40 by default, the minimum distance between
            troughs, in index values.

    output:
        new: returns a list containing the indexes where to find the throughs in the 
            input signal.
        
    example:
        x = numpy.linspace(0,2*np.pi,100)
        y = 0.5*x+numpy.sin(x)
        idx = Find_waves(y)
        
        plt.plot(x,y,'--')
        plt.plot(x[idx],y(idx),'o',label='troughs')
        plt.legend()
    
    see also:
        scipy.signal.find_peaks,
    
    TODO: ??
    NOTE: There are some verions of scipy that do not contain the function find_peaks
        therefore this function is obsolete in that case. It can be possible to change
        find_peaks to find_peaks_cwt, but I cannot warranty a good functioning.
    """
    if side == 'left':
        wave = -wave
    peak,prop = scipy.signal.find_peaks(wave,distance=dist)
    no = len(peak)
    new = []
    #print peak
    for i in range(no):
        check = wave[peak[i]]
        #print check
        if check > 0:
            new.append(peak[i])
    peak = new
    no = len(peak)
    new = [peak[0]]
    for i in range(no-1):
        #print i
        check = np.min(wave[peak[i]:peak[i+1]])
        #print check
        if check < 0:
            new.append(peak[i+1])
        if check > 0:
            #print peak[i],peak[i+1]
            if wave[peak[i]]-wave[peak[i+1]] < 0:
                if peak[i] in new:
                    new.remove(peak[i])
                new.append(peak[i+1])
        #print new
    #if new[0]>40:
    #    if wave[0] > 0:
    #        new.insert(0,0)
    #if new[-1]<len(wave)-41:
    #    if wave[-1] > 0:
    #        new.append(len(wave)-1)
    #new = np.asarray(new)
    return new